In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [4]:
admissions_sample = pd.read_pickle("data/admissions_sample.pkl")

In [5]:
admissions_sample

,subject_id,hadm_id,pos,ethnicity,ed_length,insurance,gender,age,pneumoia,diabetes,hypertension,atherosclero,renal_failure,arrhythmia
0,17524454,29928286,1,WHITE,5.700000,Other,F,60,0.0,0.0,1.0,1.0,0.0,0.0
1,10442099,25139730,1,WHITE,7.266667,Medicare,M,80,0.0,0.0,0.0,0.0,0.0,0.0
2,13761282,27810854,1,WHITE,0.000000,Other,M,85,0.0,0.0,1.0,1.0,0.0,0.0
3,17652927,28773358,1,HISPANIC/LATINO,0.000000,Other,M,46,0.0,0.0,0.0,0.0,0.0,0.0
4,10156119,23624254,1,BLACK/AFRICAN AMERICAN,4.950000,Other,F,60,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11512,17003155,26258895,0,WHITE,0.000000,Medicaid,M,62,0.0,0.0,0.0,0.0,0.0,0.0
11513,10577016,20928057,0,WHITE,16.016667,Medicaid,F,64,0.0,0.0,0.0,0.0,0.0,0.0
11514,17305471,26895604,0,WHITE,0.000000,Medicare,F,71,0.0,0.0,0.0,0.0,0.0,0.0
11515,16280277,23740212,0,WHITE,21.133333,Medicaid,F,23,0.0,0.0,0.0,0.0,0.0,0.0


## Pull Chart and Lab data only where it matches our sample patient list

In [497]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)



In [498]:
# chartevents_final.to_pickle("./data/chartevents_final.pkl")

In [499]:
# chartevents_final = pd.read_pickle("./data/chartevents_final.pkl")

In [500]:
# chartevents_final.shape

Load Lab Events table

In [6]:
# %%time

# chunksize = 3*(10 ** 6)
# counter=0
# lab_events_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
#     lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# lab_events_final = pd.concat(lab_events_sample)
# # 5k patients
# # 3min 52s

# # 10k patients
# # 4min 6s

# # 15k patients
# # 4min 27s



chunk 1 processed
chunk 2 processed
chunk 3 processed


<string>:2: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


<string>:2: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


<string>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed
CPU times: user 4min 3s, sys: 19.1 s, total: 4min 22s
Wall time: 4min 23s


In [7]:
# lab_events_final.to_pickle("./data/lab_events_final.pkl")

In [503]:
lab_events_final = pd.read_pickle("data/lab_events_final.pkl")

In [8]:
lab_events_final.shape

(6670203, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [9]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(list(admissions_sample.hadm_id))]


CPU times: user 1.8 s, sys: 658 ms, total: 2.45 s
Wall time: 2.46 s


Check how many patients have had each test done

In [10]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    8368
51265    8198
51222    8153
51301    8148
51249    8137
51248    8137
51279    8137
51250    8137
51277    8137
50912    7897
Name: subject_id, dtype: int64

Most common tests performed on patients with pos diagnosis

In [11]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    1274
50983    1272
50912    1272
51006    1271
50902    1269
51221    1268
51265    1265
50868    1258
50882    1258
51301    1253
Name: subject_id, dtype: int64

Item Black List

In [12]:
item_black_list = []
# # 50920 - test results are text in comments - need to come back and figure out how to handle this


Bonus Items To Add

In [13]:
# These test, although they have much missingness, are known to be good indicators of congestive heart failure

item_bonus_list = [51274, 51003, 50920, 50911, 51464]
# 51274 PT
# 50920 Estimated GFR (MDRD equation)
# 51003 Troponin T
# 50911 Creatine Kinase, MB Isoenzyme
# 51464 Specific Gravity


Take only the X most commonly performed tests

In [14]:
top_test_num = 20

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

itemid_sub_sample = itemid_sub_sample + item_bonus_list

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


In [15]:
# # lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))

# top_labs = lab_events_sampled.groupby(['itemid']).agg(freq=('subject_id', lambda x: len(np.unique(x))))\
# .reset_index().sort_values('freq', ascending=False).head(60)

# top_labs


In [16]:
# subset_labs['subject_id'].reset_index(drop=True).plot(kind='line')

In [17]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(1191 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())
# # print(1192 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())


In [18]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 0].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(6726 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())
# # print(6793 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())


In [19]:
# subset_labs['subject_id'].reset_index(drop=True).plot(kind='line')

## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [26]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = np.nan

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


More Data Cleaning

In [27]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' ', 'N'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy', 'SLCLOUDY'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


CPU times: user 4.4 s, sys: 101 ms, total: 4.5 s
Wall time: 4.52 s


Additional Data Cleaning

In [28]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030', '.1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['comments'].isin(['>1.050*.'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [29]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments


## Code to look into specific test values for cleaning

In [30]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'UNABLE TO REPORT']

In [31]:
# # Used to investigate bad data
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 200)

# bad_id = 51274

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates().head(100)
# # lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [32]:
def gfr_grabber(patient):
    comment = patient['comments']
    gfr_vals = []
    
    if pd.isnull(comment):
        return np.nan
    if type(comment) == str:
        if comment == '___':
            return np.nan
        
        comment = comment.replace('>75', '75')
        if ('between' in comment):
            start_low = comment.find('between')
            end_low = comment.find(' and', start_low + 8, )

            end_high = comment.find(' mL', end_low + 5, )
            
            val = (int(comment[start_low + 8 : end_low]) + int(comment[end_low + 5 : end_high])) / 2

            gfr_vals = [val, val]
        elif 'likely' in comment:
            start_point = comment.find('likely')
            end_point = comment.find(' mL', start_point + 7, )
            gfr_vals = [comment[start_point + 7 : end_point], comment[start_point + 7 : end_point]]
        else:
            start_points = [i for i in range(len(comment)) if comment.startswith('GFR = ', i)]
            end_points = list(map(lambda x: comment.find(' ', x + 6, ), start_points))

            gfr_vals = list(map(lambda x, y: comment[x + 6 : y], start_points, end_points))

    if (admissions_sample.loc[admissions_sample['subject_id'] == patient['subject_id'], 'ethnicity'] == 'BLACK/AFRICAN AMERICAN').iloc[0]:
        return int(gfr_vals[1])
    else:
        return int(gfr_vals[0])
    
lab_gfr = lab_events_sampled_sub.loc[lab_events_sampled_sub['itemid'] == 50920, ['labevent_id', 'comments', 'subject_id']]
lab_gfr['value'] = lab_gfr.apply(gfr_grabber, axis = 1)


In [33]:
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'labevent_id'].map(lambda x: lab_gfr.loc[lab_gfr['labevent_id'] == x, 'value'])
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'].map(lambda x: x.iloc[0].astype(np.float64))
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'valuenum'] = lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'value'].astype(np.float64)
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'ref_range_lower'] = 60
lab_events_sampled_sub.loc[lab_events_sampled_sub['labevent_id'].isin(lab_gfr.labevent_id), 'ref_range_upper'] = 120



/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Get Aggregate Test Values

In [34]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 315 ms, sys: 35.1 ms, total: 350 ms
Wall time: 350 ms


In [35]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [36]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     1707
max     1707
mean    1707
dtype: int64

In [37]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


51003    1397
50911    270 
50920    26  
51274    6   
51277    2   
51265    2   
51006    1   
50971    1   
51301    1   
51221    1   
Name: itemid, dtype: int64

In [38]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [39]:
lab_adm = lab_events_sampled_sub_grouped.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

lab_range_min_max_eg = lab_adm.groupby(['itemid', 'ethnicity', 'gender']).agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max_eg.reset_index(inplace = True)

lab_range_eg_dic = dict(zip(zip(lab_range_min_max_eg['itemid'], lab_range_min_max_eg['ethnicity'], lab_range_min_max_eg['gender']), zip(lab_range_min_max_eg['range_min'], lab_range_min_max_eg['range_max'])))

# # These labs have missing data for the entire ethnicity/gender combo
lab_range_eg_dic[(50911, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 10.0)
lab_range_eg_dic[(51003, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 0.01)

lab_range_eg_dic


{(50868, 'AMERICAN INDIAN/ALASKA NATIVE', 'F'): (8.0, 20.0),
 (50868, 'AMERICAN INDIAN/ALASKA NATIVE', 'M'): (8.0, 20.0),
 (50868, 'ASIAN', 'F'): (8.513513513513514, 19.37837837837838),
 (50868, 'ASIAN', 'M'): (8.575757575757576, 19.333333333333332),
 (50868, 'BLACK/AFRICAN AMERICAN', 'F'): (8.4765625, 19.4453125),
 (50868, 'BLACK/AFRICAN AMERICAN', 'M'): (8.386363636363637,
  19.488636363636363),
 (50868, 'HISPANIC/LATINO', 'F'): (8.414634146341463, 19.536585365853657),
 (50868, 'HISPANIC/LATINO', 'M'): (8.474576271186441, 19.480225988700564),
 (50868, 'OTHER', 'F'): (8.476744186046512, 19.476744186046513),
 (50868, 'OTHER', 'M'): (8.420091324200913, 19.561643835616437),
 (50868, 'UNKNOWN', 'F'): (8.438356164383562, 19.488584474885844),
 (50868, 'UNKNOWN', 'M'): (8.573770491803279, 19.39344262295082),
 (50868, 'WHITE', 'F'): (8.420539730134932, 19.514992503748125),
 (50868, 'WHITE', 'M'): (8.388558692421991, 19.540118870728083),
 (50882, 'AMERICAN INDIAN/ALASKA NATIVE', 'F'): (22.0, 3

In [40]:
# # Aggregate without splitting by Ethnicity/Gender

# lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
# lab_range_min_max.reset_index(inplace = True)

# lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
# lab_range_dic


Pivot the table so we have feature columns related to test results

In [41]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [42]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 78.6 ms, sys: 14.3 ms, total: 92.9 ms
Wall time: 91.8 ms


In [43]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [44]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()#.head(40)

             itemid
abn_percent  51221     223 
below_min    51221     223 
above_max    51221     223 
max          51221     224 
mean         51221     224 
min          51221     224 
abn_percent  51265     393 
above_max    51265     393 
below_min    51265     393 
max          51265     395 
min          51265     395 
mean         51265     395 
below_min    51222     438 
abn_percent  51222     438 
above_max    51222     438 
max          51222     438 
mean         51222     438 
min          51222     438 
above_max    51301     443 
abn_percent  51301     443 
below_min    51301     443 
mean         51301     444 
min          51301     444 
max          51301     444 
mean         51249     454 
max          51248     454 
             51249     454 
abn_percent  51249     454 
             51250     454 
max          51250     454 
             51279     454 
abn_percent  51279     454 
mean         51250     454 
above_max    51249     454 
             51248     454 


## Start Data Imputation

Copy table, retain original for Random Forest

In [45]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [46]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
115,50868,Anion Gap,Blood,Chemistry,1863-0
212,50882,Bicarbonate,Blood,Chemistry,1963-8
222,51464,Bilirubin,Urine,Hematology,5770-3
282,50893,"Calcium, Total",Blood,Chemistry,2000-8
442,50902,Chloride,Blood,Chemistry,2075-0
511,50911,"Creatine Kinase, MB Isoenzyme",Blood,Chemistry,6773-6
512,50912,Creatinine,Blood,Chemistry,2160-0
634,50920,Estimated GFR (MDRD equation),Blood,Chemistry,NaN
723,50931,Glucose,Blood,Chemistry,6777-7
761,51221,Hematocrit,Blood,Hematology,4544-3


In [47]:
lab_events_impute

min                                                                \
itemid 50868 50882 50893  50902 50911 50912 50920  50931 50960 50970 50971   
0       13.0  21.0  8.1   106.0  2.0   1.4  NaN    93.0   2.1   2.4   3.8    
1       11.0  22.0  8.5   99.0   1.0   1.1  NaN    99.0   1.8   3.8   3.5    
2       11.0  23.0  9.0   100.0 NaN    1.0   67.0  77.0   1.9   3.0   3.7    
3       16.0  20.0  9.5   106.0 NaN    0.8   75.0  92.0   1.8   3.8   4.1    
4       10.0  22.0  7.5   100.0 NaN    0.6  NaN    83.0   1.8   1.8   3.0    
...      ...   ...  ...     ...  ..    ...    ...   ...   ...   ...   ...    
8586    11.0  28.0  8.7   97.0  NaN    0.4   75.0  143.0  1.6   4.3   4.2    
8587    14.0  20.0  7.4   95.0  NaN    0.5   75.0  70.0   1.9   1.3   3.3    
8588    10.0  22.0  8.2   106.0 NaN    0.6   75.0  78.0   2.1   2.8   3.7    
8589    9.0   22.0  8.2   100.0  4.0   0.5   75.0  89.0   1.8   1.9   3.6    
8590    13.0  20.0  8.8   101.0  11.0  0.7   75.0  85.0   1.7   2.0   3.3    

                                                                            \
itemid  50983 51003 51006 51221 51222 51248 51249 51250  51265 51274 51277   
0       140.0  0.04  30.0  35.5  12.1  31.6  33.2  94.0  118.0  11.1  12.9   
1       133.0 NaN    14.0  34.1  12.5  33.5  35.2  95.0  198.0  10.5  12.8   
2       137.0 NaN    10.0  39.2  13.0  31.9  32.5  95.0  176.0  12.4  12.0   
3       140.0 NaN    11.0  39.6  12.8  30.6  32.1  95.0  352.0 NaN    13.4   
4       129.0  0.01  11.0  22.8  7.6   28.2  32.7  86.0  159.0  53.1  19.3   
...       ...  ..     ...   ...   ...   ...   ...   ...    ...  ..     ...   
8586    135.0 NaN    6.0   34.5  10.8  28.0  30.9  90.0  523.0  10.8  13.2   
8587    137.0 NaN    11.0  19.8  6.3   28.4  31.4  89.0  93.0   11.8  13.7   
8588    141.0 NaN    11.0  40.2  13.8  31.0  33.7  91.0  162.0 NaN    13.4   
8589    133.0 NaN    10.0  33.5  11.7  30.4  33.6  87.0  114.0  12.5  12.1   
8590    138.0  0.02  7.0   38.6  14.6  32.0  37.6  84.0  198.0  12.7  14.2   

                           max                                              \
itemid 51279 51301 51464 50868 50882 50893  50902 50911 50912 50920  50931   
0       3.73  3.7   0.0   19.0  26.0  8.1   108.0  36.0  1.6  NaN    150.0   
1       3.74  10.1  0.0   16.0  30.0  9.0   104.0  2.0   1.4  NaN    208.0   
2       4.08  3.3  NaN    14.0  25.0  9.6   107.0 NaN    1.1   67.0  128.0   
3       4.16  8.0   0.0   18.0  20.0  9.9   108.0 NaN    0.8   75.0  99.0    
4       2.62  2.7   0.0   12.0  23.0  7.9   109.0 NaN    0.9  NaN    159.0   
...      ...  ...   ...    ...   ...  ...     ...  ..    ...    ...   ...    
8586    3.82  8.4   0.0   15.0  30.0  9.0   102.0 NaN    0.6   75.0  340.0   
8587    2.17  6.9   0.0   19.0  31.0  9.1   109.0 NaN    1.0   75.0  153.0   
8588    4.44  5.5   0.0   17.0  27.0  8.2   109.0 NaN    0.6   75.0  93.0    
8589    3.70  6.1   0.0   15.0  30.0  8.9   111.0  5.0   0.9   75.0  149.0   
8590    4.62  12.8  0.0   16.0  27.0  9.0   106.0  11.0  0.9   75.0  173.0   

                                                                           \
itemid 50960 50970 50971  50983 51003 51006 51221 51222 51248 51249 51250   
0       2.1   2.4   4.3   143.0  1.50  33.0  43.1  14.3  32.7  34.7  95.0   
1       2.0   4.3   4.3   138.0 NaN    29.0  35.5  12.5  33.5  35.2  95.0   
2       2.1   4.0   4.4   143.0 NaN    15.0  42.4  13.8  32.1  33.7  98.0   
3       1.9   4.6   4.1   140.0 NaN    11.0  39.8  12.9  31.0  32.6  95.0   
4       2.4   2.7   4.1   138.0  0.01  21.0  25.3  8.4   29.2  33.9  88.0   
...     ...   ...   ...     ...  ..     ...   ...   ...   ...   ...   ...   
8586    1.9   5.2   4.6   139.0 NaN    10.0  35.1  11.4  29.8  32.9  91.0   
8587    4.1   3.5   4.8   143.0 NaN    26.0  32.9  10.6  29.9  33.6  93.0   
8588    2.1   2.8   4.0   142.0 NaN    17.0  46.7  15.8  31.1  34.3  92.0   
8589    2.5   3.6   4.9   143.0 NaN    24.0  43.3  14.5  31.7  35.6  93.0   
8590    1.9   2.5   3.7   141.0  0.02  1

In [48]:
lab_events_impute = lab_events_impute.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [49]:
%%time

np.random.seed(0)

for labitem in lab_range_eg_dic:
    for ind in lab_events_impute.loc[(lab_events_impute[('mean', labitem[0])].isnull()) & (lab_events_impute['ethnicity'] == labitem[1]) & (lab_events_impute['gender'] == labitem[2])].index:
        val_max = lab_range_eg_dic[labitem][1]
        val_min = lab_range_eg_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem[0])][ind]
        val_max = lab_events_impute[('max', labitem[0])][ind]
        val_mean = lab_events_impute[('mean', labitem[0])][ind]
        
        lab_events_impute[('min', labitem[0])][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem[0])][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem[0])][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 12.6 s, sys: 63.8 ms, total: 12.7 s
Wall time: 12.7 s


In [50]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute[( 'above_max', lab_itemid[0])] = lab_events_impute[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid[0])] = lab_events_impute[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid[0])] = lab_events_impute[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [51]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [52]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [53]:
lab_events_impute_KNN = lab_events_impute_KNN.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

dummy = pd.get_dummies(lab_events_impute_KNN['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('ethnicity', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)

dummy = pd.get_dummies(lab_events_impute_KNN['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('insurance', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)


lab_events_impute_KNN['gender'] = lab_events_impute_KNN['gender'].map({'M' : 0.0, 'F' : 1.0})
lab_events_impute_KNN['gender'] = pd.to_numeric(lab_events_impute_KNN['gender'])


In [54]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 1min 14s, sys: 12.4 s, total: 1min 27s
Wall time: 1min 22s


In [55]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns

In [56]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


## Standardize the Data for use in Models

In [57]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [72]:
labs_scaled = lab_events_impute.copy()
labs_scaled_KNN = lab_events_impute_KNN_imputed.copy()
labs_scaled_missing_data = lab_events_sampled_pivot.copy()

In [73]:
labs_scaled_missing_data = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [74]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender', 'insurance', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scale_cols = ~labs_scaled_KNN.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'insurance_Medicare', 'insurance_Medicaid', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scale_cols = ~labs_scaled_missing_data.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender', 'insurance', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [85]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.copy()
pos_KNN_final_test = labs_scaled_KNN.copy()
pos_final_missing_data_test = labs_scaled_missing_data.copy()


In [86]:
dummy = pd.get_dummies(pos_final_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_test = pos_final_test.drop('ethnicity', axis=1)
pos_final_test = pd.concat([pos_final_test, dummy], axis=1)

pos_final_test['gender'] = pos_final_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_test['gender'] = pd.to_numeric(pos_final_test['gender'])

dummy = pd.get_dummies(pos_final_test['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
pos_final_test = pos_final_test.drop('insurance', axis=1)
pos_final_test = pd.concat([pos_final_test, dummy], axis=1)



dummy = pd.get_dummies(pos_final_missing_data_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_missing_data_test = pos_final_missing_data_test.drop('ethnicity', axis=1)
pos_final_missing_data_test = pd.concat([pos_final_missing_data_test, dummy], axis=1)

pos_final_missing_data_test['gender'] = pos_final_missing_data_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_missing_data_test['gender'] = pd.to_numeric(pos_final_missing_data_test['gender'])

dummy = pd.get_dummies(pos_final_missing_data_test['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
pos_final_missing_data_test = pos_final_missing_data_test.drop('insurance', axis=1)
pos_final_missing_data_test = pd.concat([pos_final_missing_data_test, dummy], axis=1)


In [87]:
labitem_names = d_labitems.loc[d_labitems['itemid'].isin(itemid_sub_sample)].copy()
labitem_names['Short Name'] = labitem_names['label'] + ' - ' + labitem_names['fluid'] + ' - ' + labitem_names['category']

labitem_names = labitem_names[['itemid', 'Short Name']]
labitem_names.set_index('itemid', inplace = True)
labitem_dict = labitem_names.to_dict()['Short Name']


In [88]:
def rename_lab_col(table_name):
    num_cols = ~table_name.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'age','ed_length', 'insurance_Medicare','insurance_Medicaid', 'pneumoia', 'diabetes', 'hypertension', 'atherosclero', 'renal_failure', 'arrhythmia'])

    col_names = []
    col_names = list(table_name.loc[:, num_cols].columns)

    x, y = zip(*col_names)
    col_names = list(zip(x, list(map(labitem_dict.get, y))))

    rename_dict = dict(zip(table_name.loc[:, num_cols].columns, col_names))

    table_name.rename(columns = rename_dict, inplace = True)


In [89]:
rename_lab_col(pos_final_test)
rename_lab_col(pos_KNN_final_test)
rename_lab_col(pos_final_missing_data_test)

In [90]:
pos_final_test.to_csv('data/pos_final_test.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('data/pos_KNN_final_test.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('data/pos_final_missing_data_test.csv.gz', compression="gzip")

## Check for Multicollinearity

In [94]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [198]:
%%time

num_cols = ~pos_final_test.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos'])

labs_scaled_features = pos_final_test.loc[:, num_cols]
add_constant(labs_scaled_features)

vif_vals = [variance_inflation_factor(labs_scaled_features.values, i) for i in range(labs_scaled_features.shape[1])]
vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


CPU times: user 2min, sys: 16.8 s, total: 2min 17s
Wall time: 23.7 s


In [199]:
%%time

cols_to_drop = []
cols_to_use = ~labs_scaled_features.columns.isin(cols_to_drop)
labs_scaled_features_limited = labs_scaled_features.loc[:, cols_to_use]

while (vif_df.iloc[0, 1] > 5):
    vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features_limited.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
    cols_to_drop.append(vif_df.iloc[0, 0])
    cols_to_use = ~labs_scaled_features.columns.isin(cols_to_drop)
    
    labs_scaled_features_limited = labs_scaled_features.loc[:, cols_to_use]
    
    vif_vals = [variance_inflation_factor(labs_scaled_features_limited.values, i) for i in range(labs_scaled_features_limited.shape[1])]
    vif_df = pd.DataFrame(vif_vals, index=labs_scaled_features_limited.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
cols_to_drop  

CPU times: user 1h 17min 28s, sys: 10min 48s, total: 1h 28min 17s
Wall time: 15min 5s


[('max', 'Bilirubin - Urine - Hematology'),
 ('max', 'Troponin T - Blood - Chemistry'),
 ('mean', 'MCH - Blood - Hematology'),
 ('mean', 'Red Blood Cells - Blood - Hematology'),
 ('max', 'Glucose - Blood - Chemistry'),
 ('mean', 'Hemoglobin - Blood - Hematology'),
 ('max', 'PT - Blood - Hematology'),
 ('mean', 'MCV - Blood - Hematology'),
 ('max', 'Magnesium - Blood - Chemistry'),
 ('mean', 'Troponin T - Blood - Chemistry'),
 ('mean', 'RDW - Blood - Hematology'),
 ('max', 'Creatine Kinase, MB Isoenzyme - Blood - Chemistry'),
 ('min', 'Hemoglobin - Blood - Hematology'),
 ('max', 'Creatinine - Blood - Chemistry'),
 ('mean', 'Bilirubin - Urine - Hematology'),
 ('mean', 'MCHC - Blood - Hematology'),
 ('min', 'Hematocrit - Blood - Hematology'),
 ('mean', 'Creatinine - Blood - Chemistry'),
 ('max', 'MCH - Blood - Hematology'),
 ('mean', 'Platelet Count - Blood - Hematology'),
 ('mean', 'Estimated GFR (MDRD equation) - Blood - Chemistry'),
 ('max', 'White Blood Cells - Blood - Hematology'),
 

Again for KNN

In [200]:
%%time

num_cols_KNN = ~pos_KNN_final_test.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos'])

labs_scaled_features_KNN = pos_KNN_final_test.loc[:, num_cols_KNN]
add_constant(labs_scaled_features_KNN)

vif_vals_KNN = [variance_inflation_factor(labs_scaled_features_KNN.values, i) for i in range(labs_scaled_features_KNN.shape[1])]
vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)



CPU times: user 2min 4s, sys: 19.3 s, total: 2min 23s
Wall time: 25.1 s


In [201]:
%%time

cols_to_drop_KNN = []
cols_to_use_KNN = ~labs_scaled_features_KNN.columns.isin(cols_to_drop_KNN)
labs_scaled_features_limited_KNN = labs_scaled_features_KNN.loc[:, cols_to_use_KNN]

while (vif_df_KNN.iloc[0, 1] > 5):
    vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_limited_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
    cols_to_drop_KNN.append(vif_df_KNN.iloc[0, 0])
    cols_to_use_KNN = ~labs_scaled_features_KNN.columns.isin(cols_to_drop_KNN)
    
    labs_scaled_features_limited_KNN = labs_scaled_features_KNN.loc[:, cols_to_use_KNN]
    
    vif_vals_KNN = [variance_inflation_factor(labs_scaled_features_limited_KNN.values, i) for i in range(labs_scaled_features_limited_KNN.shape[1])]
    vif_df_KNN = pd.DataFrame(vif_vals_KNN, index=labs_scaled_features_limited_KNN.columns).reset_index().sort_values([0], ascending = False).reset_index(drop = True)
    
cols_to_drop_KNN  

CPU times: user 1h 18min 34s, sys: 10min 28s, total: 1h 29min 2s
Wall time: 15min 15s


[('max', 'Bilirubin - Urine - Hematology'),
 ('mean', 'MCH - Blood - Hematology'),
 ('mean', 'Red Blood Cells - Blood - Hematology'),
 ('mean', 'MCV - Blood - Hematology'),
 ('max', 'Glucose - Blood - Chemistry'),
 ('mean', 'Hemoglobin - Blood - Hematology'),
 ('mean', 'Estimated GFR (MDRD equation) - Blood - Chemistry'),
 ('max', 'PT - Blood - Hematology'),
 ('max', 'Magnesium - Blood - Chemistry'),
 ('mean', 'RDW - Blood - Hematology'),
 ('mean', 'Troponin T - Blood - Chemistry'),
 ('min', 'Hemoglobin - Blood - Hematology'),
 ('max', 'Creatinine - Blood - Chemistry'),
 ('max', 'MCH - Blood - Hematology'),
 ('max', 'Creatine Kinase, MB Isoenzyme - Blood - Chemistry'),
 ('mean', 'MCHC - Blood - Hematology'),
 ('mean', 'Bilirubin - Urine - Hematology'),
 ('min', 'Hematocrit - Blood - Hematology'),
 ('mean', 'Creatinine - Blood - Chemistry'),
 ('mean', 'Platelet Count - Blood - Hematology'),
 ('min', 'MCH - Blood - Hematology'),
 ('max', 'Red Blood Cells - Blood - Hematology'),
 ('max', 

Save pickles

In [213]:
pos_final_test_trimmed = labs_scaled_features.loc[:, cols_to_use].merge(pos_final_test[['subject_id', 'hadm_id', 'pos']], left_index = True, right_index = True)
pos_KNN_final_test_trimmed = labs_scaled_features_KNN.loc[:, cols_to_use_KNN].merge(pos_KNN_final_test[['subject_id', 'hadm_id', 'pos']], left_index = True, right_index = True)


In [214]:
pos_final_test_trimmed.to_csv('data/pos_final_test_trimmed.csv.gz', compression="gzip")
pos_KNN_final_test_trimmed.to_csv('data/pos_KNN_final_test_trimmed.csv.gz', compression="gzip")